In [ ]:
# Required Libraries
import pandas as pd
import numpy as np

# Step 1: Load the Dataset Paths
food_data_path = '/content/foundation_food.csv'
nutrient_data_path = '/content/food_nutrient.csv'
category_data_path = '/content/food_category.csv'
gym_data_path = '/content/megaGymDataset.csv'

# Step 2: Load the datasets with updated handling for mixed types
try:
    food_df = pd.read_csv(food_data_path)
    nutrient_df = pd.read_csv(nutrient_data_path, low_memory=False)
    category_df = pd.read_csv(category_data_path)
    gym_data = pd.read_csv(gym_data_path)
except Exception as e:
    print("Error loading datasets:", e)

# Debugging: Print column names to check for issues
print("Columns in nutrient_df:", nutrient_df.columns)
print("Columns in food_df:", food_df.columns)
print("Columns in category_df:", category_df.columns)
print("Columns in gym_data:", gym_data.columns)

# Step 3: Data Preprocessing
try:
    merged_data = pd.merge(nutrient_df, food_df, on='fdc_id', how='inner')
    print("First merge successful. Columns in merged_data:", merged_data.columns)
except KeyError as e:
    print("KeyError during merging:", e)

# Select relevant columns based on what is available
selected_columns = ['fdc_id', 'nutrient_id', 'amount', 'NDB_number', 'footnote_x']
if set(selected_columns).issubset(merged_data.columns):
    final_data = merged_data[selected_columns]
else:
    print("Some selected columns are missing.")
    final_data = None

# Rename columns for clarity if final data is prepared
if final_data is not None:
    final_data = final_data.rename(columns={
        'amount': 'nutrient_amount',
        'footnote_x': 'food_name'
    })
    print("Merged dataset is ready for analysis.")
else:
    print("Final data could not be prepared due to missing columns.")

# Convert 'nutrient_amount' to numeric using .loc to avoid SettingWithCopyWarning
if final_data is not None:
    final_data.loc[:, 'nutrient_amount'] = pd.to_numeric(final_data['nutrient_amount'], errors='coerce')

# Preprocess the Gym Exercise dataset
gym_data = gym_data[['Title', 'Type', 'BodyPart', 'Equipment', 'Level', 'Rating']]
gym_data.dropna(inplace=True)

# Step 4: Functions for Diet and Workout Recommendations

def calculate_caloric_needs(weight, height, age, gender, activity_level, goal):
    """Calculate daily caloric needs based on user profile."""
    if gender == 'male':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    else:
        bmr = 10 * weight + 6.25 * height - 5 * age - 161

    activity_multiplier = {'sedentary': 1.2, 'light': 1.375, 'moderate': 1.55, 'active': 1.725}
    tdee = bmr * activity_multiplier[activity_level]

    if goal == 'bulking':
        return tdee + 300
    elif goal == 'cutting':
        return tdee - 300
    return tdee

def recommend_foods(goal, protein_goal, final_data):
    """Recommend foods based on macronutrient needs."""
    if final_data is None:
        print("No data available for diet recommendations.")
        return None

    print("\nFiltering foods for Protein with nutrient_id == 1003 and protein_goal:", protein_goal)

    protein_data = final_data[final_data['nutrient_id'] == 1003]
    print(f"Total rows found for Protein (nutrient_id 1003): {protein_data.shape[0]}")

    # Adjust filtering to be less strict to get more results
    high_protein_foods = protein_data[protein_data['nutrient_amount'] >= (protein_goal * 0.3)]
    print(f"Rows after filtering for protein amount >= {protein_goal * 0.3}: {high_protein_foods.shape[0]}")
    return high_protein_foods.head(10)

def recommend_workouts(goal, level, equipment, gym_data):
    """Recommend workout plans based on user goal."""
    if goal == 'bulking':
        exercises = gym_data[(gym_data['Type'] == 'Strength') & (gym_data['Level'] == level)]
    elif goal == 'cutting':
        exercises = gym_data[(gym_data['Type'] == 'Cardio') & (gym_data['Level'] == level)]
    else:
        exercises = gym_data[(gym_data['Type'].isin(['Cardio', 'Strength'])) & (gym_data['Level'] == level)]

    if equipment:
        exercises = exercises[exercises['Equipment'].isin(equipment)]

    return exercises.head(10)

# Step 5: User Inputs for Testing
user_profile = {
    'age': 25,
    'weight': 70,
    'height': 175,
    'gender': 'male',
    'goal': 'bulking',
    'activity_level': 'moderate',
    'equipment': ['Dumbbells', 'Bands'],
    'level': 'Intermediate'
}

# Calculate caloric needs and adjusted protein goal
calories_needed = calculate_caloric_needs(
    user_profile['weight'],
    user_profile['height'],
    user_profile['age'],
    user_profile['gender'],
    user_profile['activity_level'],
    user_profile['goal']
)

protein_goal = calories_needed * 0.15 / 4  # Adjust protein intake to a lower threshold
diet_recommendations = recommend_foods(user_profile['goal'], protein_goal, final_data)
workout_recommendations = recommend_workouts(user_profile['goal'], user_profile['level'], user_profile['equipment'], gym_data)

# Display Recommendations
print("Caloric Needs:", calories_needed)
if diet_recommendations is not None:
    print("\nDiet Recommendations:")
    print(diet_recommendations)
print("\nWorkout Recommendations:")
print(workout_recommendations)


Columns in nutrient_df: Index(['id', 'fdc_id', 'nutrient_id', 'amount', 'data_points', 'derivation_id',
       'min', 'max', 'median', 'footnote', 'min_year_acquired'],
      dtype='object')
Columns in food_df: Index(['fdc_id', 'NDB_number', 'footnote'], dtype='object')
Columns in category_df: Index(['id', 'code', 'description'], dtype='object')
Columns in gym_data: Index(['Unnamed: 0', 'Title', 'Desc', 'Type', 'BodyPart', 'Equipment', 'Level',
       'Rating', 'RatingDesc'],
      dtype='object')
First merge successful. Columns in merged_data: Index(['id', 'fdc_id', 'nutrient_id', 'amount', 'data_points', 'derivation_id',
       'min', 'max', 'median', 'footnote_x', 'min_year_acquired', 'NDB_number',
       'footnote_y'],
      dtype='object')
Merged dataset is ready for analysis.

Filtering foods for Protein with nutrient_id == 1003 and protein_goal: 108.53671874999999
Total rows found for Protein (nutrient_id 1003): 305
Rows after filtering for protein amount >= 32.561015624999996: 

In [ ]:
# Required Libraries
import pandas as pd
import numpy as np

# Step 1: Load the Dataset Paths
food_data_path = '/content/foundation_food.csv'
nutrient_data_path = '/content/food_nutrient.csv'
category_data_path = '/content/food_category.csv'
gym_data_path = '/content/megaGymDataset.csv'

# Step 2: Load the datasets
food_df = pd.read_csv(food_data_path)
nutrient_df = pd.read_csv(nutrient_data_path, low_memory=False)
category_df = pd.read_csv(category_data_path)
gym_data = pd.read_csv(gym_data_path)

# Step 3: Merge food and nutrient datasets
merged_data = pd.merge(nutrient_df, food_df, on='fdc_id', how='inner')
merged_data.rename(columns={'amount': 'nutrient_amount', 'footnote': 'food_name'}, inplace=True)

# Debug: Print columns after merging
print("Columns after merging nutrient and food data:", merged_data.columns)

# Step 4: Merge with food category data
category_df.rename(columns={'description': 'category_description'}, inplace=True)
merged_data = pd.merge(merged_data, category_df, left_on='id', right_on='id', how='left')

# Debug: Check if 'food_name' column exists
if 'food_name' not in merged_data.columns:
    merged_data['food_name'] = np.nan  # Add 'food_name' column if missing

# Convert 'nutrient_amount' to numeric
merged_data['nutrient_amount'] = pd.to_numeric(merged_data['nutrient_amount'], errors='coerce')

# Preprocess the Gym Exercise dataset using .loc to avoid SettingWithCopyWarning
gym_data = gym_data.loc[:, ['Title', 'Type', 'BodyPart', 'Equipment', 'Level', 'Rating']].dropna()

# Step 5: Functions for Diet and Workout Recommendations
def recommend_foods(goal, protein_goal, final_data):
    """Recommend foods based on macronutrient needs."""
    protein_data = final_data[final_data['nutrient_id'] == 1003]
    high_protein_foods = protein_data[protein_data['nutrient_amount'] >= (protein_goal * 0.3)]
    return high_protein_foods[['food_name', 'nutrient_amount', 'category_description']].head(10)

def generate_weekly_workout_plan(gym_data):
    """Generate a workout plan for each day of the week."""
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    body_parts = ['Chest', 'Back', 'Legs', 'Shoulders', 'Arms', 'Abdominals', 'Full Body']
    workout_plan = {}

    for day, body_part in zip(days, body_parts):
        workouts = gym_data[gym_data['BodyPart'].str.contains(body_part, case=False, na=False)]
        if not workouts.empty:
            workout_plan[day] = workouts.sample(1).iloc[0][['Title', 'Type', 'BodyPart', 'Equipment', 'Level']].to_dict()

    return workout_plan

# Step 6: User Inputs for Recommendations
user_profile = {
    'weight': 70,       # kg
    'height': 175,      # cm
    'age': 25,          # years
    'gender': 'male',
    'goal': 'bulking',
    'activity_level': 'moderate'
}

def calculate_caloric_needs(weight, height, age, gender, activity_level, goal):
    """Calculate daily caloric needs based on user profile."""
    if gender == 'male':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    else:
        bmr = 10 * weight + 6.25 * height - 5 * age - 161

    activity_multiplier = {'sedentary': 1.2, 'light': 1.375, 'moderate': 1.55, 'active': 1.725}
    tdee = bmr * activity_multiplier[activity_level]

    if goal == 'bulking':
        return tdee + 300
    elif goal == 'cutting':
        return tdee - 300
    return tdee

# Step 7: Calculate caloric needs and protein goal
calories_needed = calculate_caloric_needs(
    user_profile['weight'],
    user_profile['height'],
    user_profile['age'],
    user_profile['gender'],
    user_profile['activity_level'],
    user_profile['goal']
)

protein_goal = calories_needed * 0.15 / 4  # Adjust protein intake
diet_recommendations = recommend_foods(user_profile['goal'], protein_goal, merged_data)
weekly_workout_plan = generate_weekly_workout_plan(gym_data)

# Step 8: Display Diet and Workout Plan
print("\n--- Diet Recommendations ---")
print(diet_recommendations)

print("\n--- Weekly Workout Plan ---")
for day, workout in weekly_workout_plan.items():
    print(f"\n{day}:")
    print(f"  Title: {workout['Title']}")
    print(f"  Type: {workout['Type']}")
    print(f"  BodyPart: {workout['BodyPart']}")
    print(f"  Equipment: {workout['Equipment']}")
    print(f"  Level: {workout['Level']}")


Columns after merging nutrient and food data: Index(['id', 'fdc_id', 'nutrient_id', 'nutrient_amount', 'data_points',
       'derivation_id', 'min', 'max', 'median', 'footnote_x',
       'min_year_acquired', 'NDB_number', 'footnote_y'],
      dtype='object')

--- Diet Recommendations ---
      food_name  nutrient_amount category_description
534         NaN             79.9                  NaN
1963        NaN             48.1                  NaN
2013        NaN             34.2                  NaN
7480        NaN             40.9                  NaN
7958        NaN             51.1                  NaN
8012        NaN             38.6                  NaN

--- Weekly Workout Plan ---

Monday:
  Title: Decline barbell bench press
  Type: Strength
  BodyPart: Chest
  Equipment: Barbell
  Level: Intermediate

Tuesday:
  Title: Smith machine inverted row
  Type: Strength
  BodyPart: Middle Back
  Equipment: Other
  Level: Intermediate

Thursday:
  Title: Upward Stretch
  Type: Stretchin

In [ ]:
# Required Libraries
import pandas as pd
import numpy as np

# Step 1: Load the Dataset Paths
food_data_path = '/content/foundation_food.csv'
nutrient_data_path = '/content/food_nutrient.csv'
category_data_path = '/content/food_category.csv'
gym_data_path = '/content/megaGymDataset.csv'

# Step 2: Load the datasets
food_df = pd.read_csv(food_data_path)
nutrient_df = pd.read_csv(nutrient_data_path, low_memory=False)
category_df = pd.read_csv(category_data_path)
gym_data = pd.read_csv(gym_data_path)

# Step 3: Merge food and nutrient datasets
merged_data = pd.merge(nutrient_df, food_df, on='fdc_id', how='inner')
merged_data.rename(columns={'amount': 'nutrient_amount', 'footnote_x': 'food_name'}, inplace=True)

# Step 4: Merge with food category data
category_df.rename(columns={'description': 'category_description'}, inplace=True)
merged_data = pd.merge(merged_data, category_df, left_on='id', right_on='id', how='left')

# Convert 'nutrient_amount' to numeric
merged_data['nutrient_amount'] = pd.to_numeric(merged_data['nutrient_amount'], errors='coerce')

# Preprocess the Gym Exercise dataset
gym_data = gym_data[['Title', 'Type', 'BodyPart', 'Equipment', 'Level', 'Rating']].dropna()

# Step 5: Functions for Diet and Workout Recommendations
def recommend_foods(goal, protein_goal, final_data):
    """Recommend foods based on macronutrient needs."""
    protein_data = final_data[final_data['nutrient_id'] == 1003]
    protein_data = protein_data[protein_data['nutrient_amount'] >= (protein_goal * 0.3)]

    # Extract food names, protein, and calories information
    food_list = protein_data[['fdc_id', 'food_name', 'nutrient_amount', 'category_description']].drop_duplicates()

    # Add calories information (nutrient_id == 1008)
    calories_data = final_data[final_data['nutrient_id'] == 1008][['fdc_id', 'nutrient_amount']]
    calories_data.rename(columns={'nutrient_amount': 'calories'}, inplace=True)

    # Merge protein data with calories data
    result = pd.merge(food_list, calories_data, on='fdc_id', how='left')
    return result.head(10)

def generate_weekly_workout_plan(gym_data):
    """Generate a workout plan with at least 5 exercises per day."""
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    body_parts = ['Chest', 'Back', 'Legs', 'Shoulders', 'Arms', 'Abdominals', 'Full Body']
    workout_plan = {}

    for day, body_part in zip(days, body_parts):
        workouts = gym_data[gym_data['BodyPart'].str.contains(body_part, case=False, na=False)]
        if not workouts.empty:
            # Select at least 5 workouts per day
            selected_workouts = workouts.sample(min(5, len(workouts)))[['Title', 'Type', 'BodyPart', 'Equipment', 'Level']]
            workout_plan[day] = selected_workouts.to_dict(orient='records')

    return workout_plan

# Step 6: User Inputs for Recommendations
user_profile = {
    'weight': 70,       # kg
    'height': 175,      # cm
    'age': 25,          # years
    'gender': 'male',
    'goal': 'bulking',
    'activity_level': 'moderate'
}

def calculate_caloric_needs(weight, height, age, gender, activity_level, goal):
    """Calculate daily caloric needs based on user profile."""
    if gender == 'male':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    else:
        bmr = 10 * weight + 6.25 * height - 5 * age - 161

    activity_multiplier = {'sedentary': 1.2, 'light': 1.375, 'moderate': 1.55, 'active': 1.725}
    tdee = bmr * activity_multiplier[activity_level]

    if goal == 'bulking':
        return tdee + 300
    elif goal == 'cutting':
        return tdee - 300
    return tdee

# Step 7: Calculate caloric needs and protein goal
calories_needed = calculate_caloric_needs(
    user_profile['weight'],
    user_profile['height'],
    user_profile['age'],
    user_profile['gender'],
    user_profile['activity_level'],
    user_profile['goal']
)

protein_goal = calories_needed * 0.15 / 4  # Adjust protein intake
diet_recommendations = recommend_foods(user_profile['goal'], protein_goal, merged_data)
weekly_workout_plan = generate_weekly_workout_plan(gym_data)

# Step 8: Display Diet and Workout Plan
print("\n--- Diet Recommendations ---")
print(diet_recommendations)

print("\n--- Weekly Workout Plan ---")
for day, workouts in weekly_workout_plan.items():
    print(f"\n{day}:")
    for workout in workouts:
        print(f"  Title: {workout['Title']}")
        print(f"  Type: {workout['Type']}")
        print(f"  BodyPart: {workout['BodyPart']}")
        print(f"  Equipment: {workout['Equipment']}")
        print(f"  Level: {workout['Level']}")
        print("-" * 20)



--- Diet Recommendations ---
    fdc_id food_name  nutrient_amount category_description  calories
0   323793       NaN             79.9                  NaN     376.0
1   329490       NaN             48.1                  NaN     575.0
2   329716       NaN             34.2                  NaN     654.0
3   749420       NaN             40.9                  NaN     500.0
4  1104705       NaN             51.1                  NaN     366.0
5  1104766       NaN             38.6                  NaN     452.0

--- Weekly Workout Plan ---

Monday:
  Title: Weighted Push-Up
  Type: Strength
  BodyPart: Chest
  Equipment: Other
  Level: Intermediate
--------------------
  Title: Wide-grip bench press
  Type: Strength
  BodyPart: Chest
  Equipment: Barbell
  Level: Beginner
--------------------
  Title: Chain Press
  Type: Powerlifting
  BodyPart: Chest
  Equipment: Other
  Level: Beginner
--------------------
  Title: One-Arm Kettlebell Floor Press
  Type: Strength
  BodyPart: Chest
  Equip

In [ ]:
# Required Libraries
import pandas as pd
import numpy as np

# Step 1: Load the ABBREV.csv dataset
abbrev_path = '/content/ABBREV.csv'
abbrev_df = pd.read_csv(abbrev_path)

# Clean the ABBREV dataset
abbrev_df['Energ_Kcal'] = pd.to_numeric(abbrev_df['Energ_Kcal'], errors='coerce')
abbrev_df['Protein_(g)'] = pd.to_numeric(abbrev_df['Protein_(g)'], errors='coerce')
abbrev_df['Carbohydrt_(g)'] = pd.to_numeric(abbrev_df['Carbohydrt_(g)'], errors='coerce')
abbrev_df['Lipid_Tot_(g)'] = pd.to_numeric(abbrev_df['Lipid_Tot_(g)'], errors='coerce')

# Select relevant columns for food recommendations
abbrev_df = abbrev_df[['Shrt_Desc', 'Energ_Kcal', 'Protein_(g)', 'Carbohydrt_(g)', 'Lipid_Tot_(g)']]

# Step 2: Load the Indian cuisines dataset
indian_cuisines_path = '/content/indian_cuisines.csv'
indian_cuisines_df = pd.read_csv(indian_cuisines_path)

# Clean the Indian cuisines dataset
indian_cuisines_df['calories'] = indian_cuisines_df['calories'].str.replace(' kcal', '').astype(float)
indian_cuisines_df['proteins'] = indian_cuisines_df['proteins'].str.replace('g', '').astype(float)
indian_cuisines_df['carbohydrates'] = indian_cuisines_df['carbohydrates'].str.replace('g', '').astype(float)
indian_cuisines_df['fats'] = indian_cuisines_df['fats'].str.replace('g', '').astype(float)

# Step 3: Load the Gym Exercise dataset
gym_data_path = '/content/megaGymDataset.csv'
gym_data = pd.read_csv(gym_data_path)

# Preprocess the Gym Exercise dataset
gym_data = gym_data[['Title', 'Type', 'BodyPart', 'Equipment', 'Level', 'Rating']].dropna()

# Step 4: Define Functions for Diet and Workout Recommendations

def recommend_abbrev_foods(goal):
    """Recommend foods from ABBREV.csv based on the goal."""
    if goal == 'high_protein':
        return abbrev_df.sort_values(by='Protein_(g)', ascending=False).head(10)
    elif goal == 'low_carbs':
        return abbrev_df.sort_values(by='Carbohydrt_(g)').head(10)
    elif goal == 'high_fats':
        return abbrev_df.sort_values(by='Lipid_Tot_(g)', ascending=False).head(10)
    else:
        return abbrev_df.sort_values(by='Energ_Kcal', ascending=False).head(10)

def recommend_indian_foods(goal):
    """Recommend foods from the Indian cuisines dataset based on the goal."""
    if goal == 'high_protein':
        return indian_cuisines_df.sort_values(by='proteins', ascending=False).head(10)
    elif goal == 'low_carbs':
        return indian_cuisines_df.sort_values(by='carbohydrates').head(10)
    elif goal == 'high_fats':
        return indian_cuisines_df.sort_values(by='fats', ascending=False).head(10)
    else:
        return indian_cuisines_df.sort_values(by='calories', ascending=False).head(10)

def generate_weekly_workout_plan(gym_data):
    """Generate a workout plan with at least 5 exercises per day."""
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    body_parts = ['Chest', 'Back', 'Legs', 'Shoulders', 'Arms', 'Abdominals', 'Full Body']
    workout_plan = {}

    for day, body_part in zip(days, body_parts):
        workouts = gym_data[gym_data['BodyPart'].str.contains(body_part, case=False, na=False)]
        if not workouts.empty:
            selected_workouts = workouts.sample(min(5, len(workouts)))[['Title', 'Type', 'BodyPart', 'Equipment', 'Level']]
            workout_plan[day] = selected_workouts.to_dict(orient='records')

    return workout_plan

# Step 5: Get User Inputs
user_goal = input("Enter your fitness goal (high_protein, low_carbs, high_fats, high_calories): ").strip().lower()

# Step 6: Generate Diet Recommendations
print("\n--- Recommendations from ABBREV Dataset ---")
abbrev_recommendations = recommend_abbrev_foods(user_goal)
print(abbrev_recommendations)

print("\n--- Recommendations from Indian Cuisines Dataset ---")
indian_recommendations = recommend_indian_foods(user_goal)
print(indian_recommendations)

# Step 7: Generate Workout Plan
print("\n--- Weekly Workout Plan ---")
weekly_workout_plan = generate_weekly_workout_plan(gym_data)
for day, workouts in weekly_workout_plan.items():
    print(f"\n{day}:")
    for workout in workouts:
        print(f"  Title: {workout['Title']}, Type: {workout['Type']}, BodyPart: {workout['BodyPart']}, Equipment: {workout['Equipment']}, Level: {workout['Level']}")
        print("-" * 20)


Enter your fitness goal (high_protein, low_carbs, high_fats, high_calories): high_protein

--- Recommendations from ABBREV Dataset ---
                                             Shrt_Desc  Energ_Kcal  \
4858                               SOY PROTEIN ISOLATE         335   
5009                           SOY PROT ISOLATE,K TYPE         321   
6204                          GELATINS,DRY PDR,UNSWTND         335   
216             EGG,WHITE,DRIED,STABILIZED,GLUCOSE RED         357   
8327  SEAL,BEARDED (OOGRUK),MEAT,DRIED (ALASKA NATIVE)         351   
124         EGG,WHITE,DRIED,PDR,STABILIZED,GLUCOSE RED         376   
151                                    EGG,WHITE,DRIED         382   
4171                       BEVERAGES,PROT PDR WHEY BSD         352   
8409    STEELHEAD TROUT,DRIED,FLESH (SHOSHONE BANNOCK)         382   
123      EGG,WHITE,DRIED,FLAKES,STABILIZED,GLUCOSE RED         351   

      Protein_(g)  Carbohydrt_(g)  Lipid_Tot_(g)  
4858        88.32            0.00          

In [ ]:
# Required Libraries
import pandas as pd
import numpy as np

# Step 1: Load the ABBREV.csv Dataset
abbrev_path = '/content/ABBREV.csv'
abbrev_df = pd.read_csv(abbrev_path)

# Clean the ABBREV dataset
abbrev_df['Energ_Kcal'] = pd.to_numeric(abbrev_df['Energ_Kcal'], errors='coerce')
abbrev_df['Protein_(g)'] = pd.to_numeric(abbrev_df['Protein_(g)'], errors='coerce')
abbrev_df['Carbohydrt_(g)'] = pd.to_numeric(abbrev_df['Carbohydrt_(g)'], errors='coerce')
abbrev_df['Lipid_Tot_(g)'] = pd.to_numeric(abbrev_df['Lipid_Tot_(g)'], errors='coerce')
abbrev_df = abbrev_df[['Shrt_Desc', 'Energ_Kcal', 'Protein_(g)', 'Carbohydrt_(g)', 'Lipid_Tot_(g)']]

# Load the Indian cuisines dataset
indian_cuisines_path = '/content/indian_cuisines.csv'
indian_cuisines_df = pd.read_csv(indian_cuisines_path)
indian_cuisines_df['calories'] = indian_cuisines_df['calories'].str.replace(' kcal', '').astype(float)
indian_cuisines_df['proteins'] = indian_cuisines_df['proteins'].str.replace('g', '').astype(float)
indian_cuisines_df['carbohydrates'] = indian_cuisines_df['carbohydrates'].str.replace('g', '').astype(float)
indian_cuisines_df['fats'] = indian_cuisines_df['fats'].str.replace('g', '').astype(float)

# Load the Gym Exercise dataset
gym_data_path = '/content/megaGymDataset.csv'
gym_data = pd.read_csv(gym_data_path)
gym_data = gym_data[['Title', 'Type', 'BodyPart', 'Equipment', 'Level', 'Rating']].dropna()

# Step 2: Define Functions for Diet and Workout Recommendations

def calculate_bmr(weight, height, age, gender):
    """Calculate Basal Metabolic Rate (BMR) based on gender."""
    if gender == 'male':
        return 10 * weight + 6.25 * height - 5 * age + 5
    else:
        return 10 * weight + 6.25 * height - 5 * age - 161

def calculate_tdee(bmr, activity_level):
    """Calculate Total Daily Energy Expenditure (TDEE) based on activity level."""
    activity_multiplier = {
        'sedentary': 1.2,
        'light': 1.375,
        'moderate': 1.55,
        'active': 1.725
    }
    return bmr * activity_multiplier.get(activity_level, 1.2)

def calculate_lean_body_mass(weight, body_fat_percentage):
    """Calculate lean body mass."""
    return weight * (1 - (body_fat_percentage / 100))

def calculate_protein_intake(lean_body_mass, goal):
    """Calculate protein intake based on fitness goal."""
    if goal == 'bulking':
        return lean_body_mass * 2.2
    elif goal == 'cutting':
        return lean_body_mass * 2.7
    else:
        return lean_body_mass * 1.8

# Function to provide diverse food recommendations from ABBREV.csv
def recommend_diverse_foods(abbrev_df):
    """Provide a variety of foods from different categories."""
    dairy = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('CHEESE|MILK|YOGURT', case=False)].head(5)
    meats = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('BEEF|CHICKEN|TURKEY|PORK', case=False)].head(5)
    vegetables = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('VEG|CARROT|SPINACH|BROCCOLI|TOMATO', case=False)].head(5)
    legumes_nuts = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('BEAN|LENTIL|NUT', case=False)].head(5)
    grains = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('RICE|OAT|WHEAT|CORN', case=False)].head(5)
    return dairy, meats, vegetables, legumes_nuts, grains

def recommend_indian_foods(indian_cuisines_df, goal):
    """Recommend foods from Indian cuisines dataset based on the goal."""
    if goal == 'high_protein':
        return indian_cuisines_df.sort_values(by='proteins', ascending=False).head(10)
    elif goal == 'low_carbs':
        return indian_cuisines_df.sort_values(by='carbohydrates').head(10)
    elif goal == 'high_fats':
        return indian_cuisines_df.sort_values(by='fats', ascending=False).head(10)
    else:
        return indian_cuisines_df.sort_values(by='calories', ascending=False).head(10)

def generate_weekly_workout_plan(gym_data):
    """Generate a workout plan with at least 5 exercises per day."""
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    body_parts = ['Chest', 'Back', 'Legs', 'Shoulders', 'Arms', 'Abdominals', 'Full Body']
    workout_plan = {}
    for day, body_part in zip(days, body_parts):
        workouts = gym_data[gym_data['BodyPart'].str.contains(body_part, case=False, na=False)]
        if not workouts.empty:
            selected_workouts = workouts.sample(min(5, len(workouts)))[['Title', 'Type', 'BodyPart', 'Equipment', 'Level']]
            workout_plan[day] = selected_workouts.to_dict(orient='records')
    return workout_plan

# Step 3: Get User Inputs
height = float(input("Enter your height in cm: "))
weight = float(input("Enter your weight in kg: "))
age = int(input("Enter your age: "))
gender = input("Enter your gender (male/female): ").lower()
activity_level = input("Enter your activity level (sedentary, light, moderate, active): ").lower()
body_fat_percentage = float(input("Enter your body fat percentage: "))
goal = input("Enter your goal (bulking, cutting, maintaining): ").lower()

# Step 4: Calculate BMR, TDEE, and Protein Requirements
bmr = calculate_bmr(weight, height, age, gender)
tdee = calculate_tdee(bmr, activity_level)
lean_body_mass = calculate_lean_body_mass(weight, body_fat_percentage)
protein_goal = calculate_protein_intake(lean_body_mass, goal)

# Step 5: Generate Recommendations
dairy, meats, vegetables, legumes_nuts, grains = recommend_diverse_foods(abbrev_df)
indian_food_recommendations = recommend_indian_foods(indian_cuisines_df, goal)

print("\n--- Dairy Recommendations ---")
print(dairy)
print("\n--- Meat Recommendations ---")
print(meats)
print("\n--- Vegetable Recommendations ---")
print(vegetables)
print("\n--- Legumes & Nuts Recommendations ---")
print(legumes_nuts)
print("\n--- Grains Recommendations ---")
print(grains)
print("\n--- Indian Cuisine Recommendations ---")
print(indian_food_recommendations)

# Step 6: Generate Workout Plan
workout_plan = generate_weekly_workout_plan(gym_data)
print("\n--- Weekly Workout Plan ---")
for day, exercises in workout_plan.items():
    print(f"\n{day}:")
    for exercise in exercises:
        print(f"  Title: {exercise['Title']}, Type: {exercise['Type']}, BodyPart: {exercise['BodyPart']}, Equipment: {exercise['Equipment']}, Level: {exercise['Level']}")
        print("-" * 20)


KeyboardInterrupt: Interrupted by user

In [ ]:
#with cleaned NaN values before preprocessing
# Required Libraries
import pandas as pd
import numpy as np

# Step 1: Load and Preprocess the Datasets
def load_and_clean_abbrev_data(path):
    """Load and clean the ABBREV.csv dataset."""
    abbrev_df = pd.read_csv(path)
    abbrev_df['Energ_Kcal'] = pd.to_numeric(abbrev_df['Energ_Kcal'], errors='coerce')
    abbrev_df['Protein_(g)'] = pd.to_numeric(abbrev_df['Protein_(g)'], errors='coerce')
    abbrev_df['Carbohydrt_(g)'] = pd.to_numeric(abbrev_df['Carbohydrt_(g)'], errors='coerce')
    abbrev_df['Lipid_Tot_(g)'] = pd.to_numeric(abbrev_df['Lipid_Tot_(g)'], errors='coerce')
    abbrev_df.dropna(subset=['Shrt_Desc', 'Energ_Kcal', 'Protein_(g)', 'Carbohydrt_(g)', 'Lipid_Tot_(g)'], inplace=True)
    return abbrev_df

def load_and_clean_indian_cuisine_data(path):
    """Load and clean the Indian cuisines dataset."""
    indian_cuisines_df = pd.read_csv(path)
    indian_cuisines_df['calories'] = indian_cuisines_df['calories'].str.replace(' kcal', '').astype(float)
    indian_cuisines_df['proteins'] = indian_cuisines_df['proteins'].str.replace('g', '').astype(float)
    indian_cuisines_df['carbohydrates'] = indian_cuisines_df['carbohydrates'].str.replace('g', '').astype(float)
    indian_cuisines_df['fats'] = indian_cuisines_df['fats'].str.replace('g', '').astype(float)
    indian_cuisines_df.dropna(subset=['cuisine', 'calories', 'proteins', 'carbohydrates', 'fats'], inplace=True)
    return indian_cuisines_df

def load_and_clean_gym_data(path):
    """Load and clean the Gym Exercise dataset."""
    gym_data = pd.read_csv(path)
    gym_data = gym_data[['Title', 'Type', 'BodyPart', 'Equipment', 'Level', 'Rating']].dropna()
    return gym_data

# Load datasets
abbrev_df = load_and_clean_abbrev_data('/content/ABBREV.csv')
indian_cuisines_df = load_and_clean_indian_cuisine_data('/content/indian_cuisines.csv')
gym_data = load_and_clean_gym_data('/content/megaGymDataset.csv')

# Step 2: Define Functions for Recommendations

def calculate_bmr(weight, height, age, gender):
    """Calculate Basal Metabolic Rate (BMR)."""
    if gender == 'male':
        return 10 * weight + 6.25 * height - 5 * age + 5
    else:
        return 10 * weight + 6.25 * height - 5 * age - 161

def calculate_tdee(bmr, activity_level):
    """Calculate Total Daily Energy Expenditure (TDEE)."""
    activity_multiplier = {'sedentary': 1.2, 'light': 1.375, 'moderate': 1.55, 'active': 1.725}
    return bmr * activity_multiplier.get(activity_level, 1.2)

def calculate_lean_body_mass(weight, body_fat_percentage):
    """Calculate lean body mass."""
    return weight * (1 - (body_fat_percentage / 100))

def calculate_protein_intake(lean_body_mass, goal):
    """Calculate protein intake based on fitness goal."""
    if goal == 'bulking':
        return lean_body_mass * 2.2
    elif goal == 'cutting':
        return lean_body_mass * 2.7
    else:
        return lean_body_mass * 1.8

def recommend_diverse_foods(abbrev_df):
    """Provide diverse food recommendations."""
    dairy = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('CHEESE|MILK|YOGURT', case=False)].head(5)
    meats = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('BEEF|CHICKEN|TURKEY|PORK', case=False)].head(5)
    vegetables = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('VEG|CARROT|SPINACH|BROCCOLI|TOMATO', case=False)].head(5)
    legumes_nuts = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('BEAN|LENTIL|NUT', case=False)].head(5)
    grains = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('RICE|OAT|WHEAT|CORN', case=False)].head(5)
    return dairy, meats, vegetables, legumes_nuts, grains

def recommend_indian_foods(indian_cuisines_df, goal):
    """Recommend Indian foods based on fitness goal."""
    if goal == 'high_protein':
        return indian_cuisines_df.sort_values(by='proteins', ascending=False).head(10)
    elif goal == 'low_carbs':
        return indian_cuisines_df.sort_values(by='carbohydrates').head(10)
    elif goal == 'high_fats':
        return indian_cuisines_df.sort_values(by='fats', ascending=False).head(10)
    else:
        return indian_cuisines_df.sort_values(by='calories', ascending=False).head(10)

def generate_weekly_workout_plan(gym_data):
    """Generate a workout plan with at least 5 exercises per day."""
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    body_parts = ['Chest', 'Back', 'Legs', 'Shoulders', 'Arms', 'Abdominals', 'Full Body']
    workout_plan = {}
    for day, body_part in zip(days, body_parts):
        workouts = gym_data[gym_data['BodyPart'].str.contains(body_part, case=False, na=False)]
        if not workouts.empty:
            selected_workouts = workouts.sample(min(5, len(workouts)))[['Title', 'Type', 'BodyPart', 'Equipment', 'Level']]
            workout_plan[day] = selected_workouts.to_dict(orient='records')
    return workout_plan

# Step 3: Get User Inputs
height = float(input("Enter your height in cm: "))
weight = float(input("Enter your weight in kg: "))
age = int(input("Enter your age: "))
gender = input("Enter your gender (male/female): ").lower()
activity_level = input("Enter your activity level (sedentary, light, moderate, active): ").lower()
body_fat_percentage = float(input("Enter your body fat percentage: "))
goal = input("Enter your goal (bulking, cutting, maintaining): ").lower()

# Step 4: Calculate Requirements and Generate Recommendations
bmr = calculate_bmr(weight, height, age, gender)
tdee = calculate_tdee(bmr, activity_level)
lean_body_mass = calculate_lean_body_mass(weight, body_fat_percentage)
protein_goal = calculate_protein_intake(lean_body_mass, goal)

# Generate Food and Workout Recommendations
dairy, meats, vegetables, legumes_nuts, grains = recommend_diverse_foods(abbrev_df)
indian_food_recommendations = recommend_indian_foods(indian_cuisines_df, 'high_protein')
workout_plan = generate_weekly_workout_plan(gym_data)

# Display Results
print("\n--- Diverse Food Recommendations ---")
print("Dairy:", dairy)
print("Meats:", meats)
print("Vegetables:", vegetables)
print("Legumes & Nuts:", legumes_nuts)
print("Grains:", grains)

print("\n--- Indian Cuisine Recommendations ---")
print(indian_food_recommendations)

print("\n--- Weekly Workout Plan ---")
for day, exercises in workout_plan.items():
    print(f"\n{day}:")
    for exercise in exercises:
        print(f"  {exercise['Title']} | {exercise['Type']} | {exercise['BodyPart']} | {exercise['Equipment']} | {exercise['Level']}")


Enter your height in cm: 172
Enter your weight in kg: 55.5
Enter your age: 19
Enter your gender (male/female): male
Enter your activity level (sedentary, light, moderate, active): moderate
Enter your body fat percentage: 12.5
Enter your goal (bulking, cutting, maintaining): cutting

--- Diverse Food Recommendations ---
Dairy:    index  NDB_No         Shrt_Desc  Water_(g)  Energ_Kcal  Protein_(g)  \
3      3    1004       CHEESE,BLUE      42.41         353        21.40   
4      4    1005      CHEESE,BRICK      41.11         371        23.24   
5      5    1006       CHEESE,BRIE      48.42         334        20.75   
6      6    1007  CHEESE,CAMEMBERT      51.80         300        19.80   
7      7    1008    CHEESE,CARAWAY      39.28         376        25.18   

   Lipid_Tot_(g)  Ash_(g)  Carbohydrt_(g)  Fiber_TD_(g)  ...  Vit_K_(µg)  \
3          28.74     5.11            2.34           0.0  ...         2.4   
4          29.68     3.18            2.79           0.0  ...         2.5   

In [ ]:
#mostly final code

# Required Libraries
import pandas as pd
import numpy as np

# Step 1: Load and Preprocess the Datasets
def load_and_clean_abbrev_data(path):
    """Load and clean the ABBREV.csv dataset."""
    abbrev_df = pd.read_csv(path)
    abbrev_df['Energ_Kcal'] = pd.to_numeric(abbrev_df['Energ_Kcal'], errors='coerce')
    abbrev_df['Protein_(g)'] = pd.to_numeric(abbrev_df['Protein_(g)'], errors='coerce')
    abbrev_df['Carbohydrt_(g)'] = pd.to_numeric(abbrev_df['Carbohydrt_(g)'], errors='coerce')
    abbrev_df['Water_(g)'] = pd.to_numeric(abbrev_df['Water_(g)'], errors='coerce')
    abbrev_df.dropna(subset=['Shrt_Desc', 'Energ_Kcal', 'Protein_(g)', 'Carbohydrt_(g)', 'Water_(g)'], inplace=True)
    return abbrev_df

def load_and_clean_indian_cuisine_data(path):
    """Load and clean the Indian cuisines dataset."""
    indian_cuisines_df = pd.read_csv(path)
    indian_cuisines_df['calories'] = indian_cuisines_df['calories'].str.replace(' kcal', '').astype(float)
    indian_cuisines_df['proteins'] = indian_cuisines_df['proteins'].str.replace('g', '').astype(float)
    indian_cuisines_df['carbohydrates'] = indian_cuisines_df['carbohydrates'].str.replace('g', '').astype(float)
    indian_cuisines_df['fats'] = indian_cuisines_df['fats'].str.replace('g', '').astype(float)
    indian_cuisines_df.dropna(subset=['cuisine', 'calories', 'proteins', 'carbohydrates', 'fats'], inplace=True)
    return indian_cuisines_df

def load_and_clean_gym_data(path):
    """Load and clean the Gym Exercise dataset."""
    gym_data = pd.read_csv(path)
    gym_data = gym_data[['Title', 'Type', 'BodyPart', 'Equipment', 'Level', 'Rating']].dropna()
    return gym_data

# Load datasets
abbrev_df = load_and_clean_abbrev_data('/content/ABBREV.csv')
indian_cuisines_df = load_and_clean_indian_cuisine_data('/content/indian_cuisines.csv')
gym_data = load_and_clean_gym_data('/content/megaGymDataset.csv')

# Step 2: Define Functions for Recommendations

def calculate_bmr(weight, height, age, gender):
    """Calculate Basal Metabolic Rate (BMR)."""
    if gender == 'male':
        return 10 * weight + 6.25 * height - 5 * age + 5
    else:
        return 10 * weight + 6.25 * height - 5 * age - 161

def calculate_tdee(bmr, activity_level):
    """Calculate Total Daily Energy Expenditure (TDEE)."""
    activity_multiplier = {'sedentary': 1.2, 'light': 1.375, 'moderate': 1.55, 'active': 1.725}
    return bmr * activity_multiplier.get(activity_level, 1.2)

def calculate_lean_body_mass(weight, body_fat_percentage):
    """Calculate lean body mass."""
    return weight * (1 - (body_fat_percentage / 100))

def calculate_protein_intake(lean_body_mass, goal):
    """Calculate protein intake based on fitness goal."""
    if goal == 'bulking':
        return lean_body_mass * 2.2
    elif goal == 'cutting':
        return lean_body_mass * 2.7
    else:
        return lean_body_mass * 1.8

def recommend_diverse_foods(abbrev_df):
    """Provide detailed food recommendations with necessary columns."""
    dairy = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('CHEESE|MILK|YOGURT', case=False)].head(5)
    meats = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('BEEF|CHICKEN|TURKEY|PORK', case=False)].head(5)
    vegetables = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('VEG|CARROT|SPINACH|BROCCOLI|TOMATO', case=False)].head(5)
    legumes_nuts = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('BEAN|LENTIL|NUT', case=False)].head(5)
    grains = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('RICE|OAT|WHEAT|CORN', case=False)].head(5)

    cols_to_display = ['Shrt_Desc', 'Water_(g)', 'Energ_Kcal', 'Protein_(g)', 'Carbohydrt_(g)']
    print("\n--- Dairy ---")
    print(dairy[cols_to_display])
    print("\n--- Meats ---")
    print(meats[cols_to_display])
    print("\n--- Vegetables ---")
    print(vegetables[cols_to_display])
    print("\n--- Legumes & Nuts ---")
    print(legumes_nuts[cols_to_display])
    print("\n--- Grains ---")
    print(grains[cols_to_display])

def recommend_indian_foods(indian_cuisines_df, goal):
    """Recommend Indian foods based on fitness goal."""
    if goal == 'high_protein':
        return indian_cuisines_df.sort_values(by='proteins', ascending=False).head(10)
    elif goal == 'low_carbs':
        return indian_cuisines_df.sort_values(by='carbohydrates').head(10)
    elif goal == 'high_fats':
        return indian_cuisines_df.sort_values(by='fats', ascending=False).head(10)
    else:
        return indian_cuisines_df.sort_values(by='calories', ascending=False).head(10)

def generate_weekly_workout_plan(gym_data):
    """Generate a workout plan with at least 5 exercises per day."""
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    body_parts = ['Chest', 'Back', 'Legs', 'Shoulders', 'Arms', 'Abdominals', 'Full Body']
    workout_plan = {}
    for day, body_part in zip(days, body_parts):
        workouts = gym_data[gym_data['BodyPart'].str.contains(body_part, case=False, na=False)]
        if not workouts.empty:
            selected_workouts = workouts.sample(min(5, len(workouts)))[['Title', 'Type', 'BodyPart', 'Equipment', 'Level']]
            workout_plan[day] = selected_workouts.to_dict(orient='records')
    return workout_plan

# Step 3: Get User Inputs
height = float(input("Enter your height in cm: "))
weight = float(input("Enter your weight in kg: "))
age = int(input("Enter your age: "))
gender = input("Enter your gender (male/female): ").lower()
activity_level = input("Enter your activity level (sedentary, light, moderate, active): ").lower()
body_fat_percentage = float(input("Enter your body fat percentage: "))
goal = input("Enter your goal (bulking, cutting, maintaining): ").lower()

# Step 4: Calculate Requirements and Generate Recommendations
bmr = calculate_bmr(weight, height, age, gender)
tdee = calculate_tdee(bmr, activity_level)
lean_body_mass = calculate_lean_body_mass(weight, body_fat_percentage)
protein_goal = calculate_protein_intake(lean_body_mass, goal)

# Generate Recommendations
print("\n--- Diverse Food Recommendations ---")
recommend_diverse_foods(abbrev_df)

indian_food_recommendations = recommend_indian_foods(indian_cuisines_df, goal)
print("\n--- Indian Cuisine Recommendations ---")
print(indian_food_recommendations)

workout_plan = generate_weekly_workout_plan(gym_data)
print("\n--- Weekly Workout Plan ---")
for day, exercises in workout_plan.items():
    print(f"\n{day}:")
    for exercise in exercises:
        print(f"  {exercise['Title']} | {exercise['Type']} | {exercise['BodyPart']} | {exercise['Equipment']} | {exercise['Level']}")


Enter your height in cm: 172
Enter your weight in kg: 55.5
Enter your age: 19
Enter your gender (male/female): male
Enter your activity level (sedentary, light, moderate, active): moderate
Enter your body fat percentage: 12.5
Enter your goal (bulking, cutting, maintaining): bulking

--- Diverse Food Recommendations ---

--- Dairy ---
          Shrt_Desc  Water_(g)  Energ_Kcal  Protein_(g)  Carbohydrt_(g)
3       CHEESE,BLUE      42.41         353        21.40            2.34
4      CHEESE,BRICK      41.11         371        23.24            2.79
5       CHEESE,BRIE      48.42         334        20.75            0.45
6  CHEESE,CAMEMBERT      51.80         300        19.80            0.46
7    CHEESE,CARAWAY      39.28         376        25.18            3.06

--- Meats ---
                                 Shrt_Desc  Water_(g)  Energ_Kcal  \
129                EGG,TURKEY,WHL,FRSH,RAW      72.50         171   
319                 BABYFOOD,MEAT,PORK,STR      78.40         124   
327       

In [ ]:
# Required Libraries
import pandas as pd
import numpy as np

# Step 1: Load and Preprocess the Datasets
def load_and_clean_abbrev_data(path):
    abbrev_df = pd.read_csv(path)
    abbrev_df['Energ_Kcal'] = pd.to_numeric(abbrev_df['Energ_Kcal'], errors='coerce')
    abbrev_df['Protein_(g)'] = pd.to_numeric(abbrev_df['Protein_(g)'], errors='coerce')
    abbrev_df['Carbohydrt_(g)'] = pd.to_numeric(abbrev_df['Carbohydrt_(g)'], errors='coerce')
    abbrev_df['Water_(g)'] = pd.to_numeric(abbrev_df['Water_(g)'], errors='coerce')
    abbrev_df.dropna(subset=['Shrt_Desc', 'Energ_Kcal', 'Protein_(g)', 'Carbohydrt_(g)', 'Water_(g)'], inplace=True)
    return abbrev_df

def load_and_clean_indian_cuisine_data(path):
    indian_cuisines_df = pd.read_csv(path)
    indian_cuisines_df['calories'] = indian_cuisines_df['calories'].str.replace(' kcal', '').astype(float)
    indian_cuisines_df['proteins'] = indian_cuisines_df['proteins'].str.replace('g', '').astype(float)
    indian_cuisines_df['carbohydrates'] = indian_cuisines_df['carbohydrates'].str.replace('g', '').astype(float)
    indian_cuisines_df['fats'] = indian_cuisines_df['fats'].str.replace('g', '').astype(float)
    indian_cuisines_df.dropna(subset=['cuisine', 'calories', 'proteins', 'carbohydrates', 'fats'], inplace=True)
    return indian_cuisines_df

def load_and_clean_gym_data(path):
    gym_data = pd.read_csv(path)
    gym_data = gym_data[['Title', 'Type', 'BodyPart', 'Equipment', 'Level']].dropna()
    return gym_data

# Load datasets
abbrev_df = load_and_clean_abbrev_data('/content/ABBREV.csv')
indian_cuisines_df = load_and_clean_indian_cuisine_data('/content/indian_cuisines.csv')
gym_data = load_and_clean_gym_data('/content/megaGymDataset.csv')

# Step 2: Define the Intelligent Agent class
class FitnessAgent:
    def __init__(self, height, weight, age, gender, activity_level, body_fat_percentage, goal):
        self.height = height
        self.weight = weight
        self.age = age
        self.gender = gender
        self.activity_level = activity_level
        self.body_fat_percentage = body_fat_percentage
        self.goal = goal

    def calculate_bmr(self):
        if self.gender == 'male':
            return 10 * self.weight + 6.25 * self.height - 5 * self.age + 5
        else:
            return 10 * self.weight + 6.25 * self.height - 5 * self.age - 161

    def calculate_tdee(self, bmr):
        activity_multiplier = {'sedentary': 1.2, 'light': 1.375, 'moderate': 1.55, 'active': 1.725}
        return bmr * activity_multiplier.get(self.activity_level, 1.2)

    def calculate_lean_body_mass(self):
        return self.weight * (1 - (self.body_fat_percentage / 100))

    def calculate_protein_intake(self, lean_body_mass):
        if self.goal == 'bulking':
            return lean_body_mass * 2.2
        elif self.goal == 'cutting':
            return lean_body_mass * 2.7
        else:
            return lean_body_mass * 1.8

    def generate_diet_recommendations(self):
        dairy, meats, vegetables, grains = recommend_diverse_foods(abbrev_df)
        indian_food_recommendations = recommend_indian_foods(indian_cuisines_df, self.goal)
        return dairy, meats, vegetables, grains, indian_food_recommendations

    def generate_workout_plan(self):
        return generate_weekly_workout_plan(gym_data)

# Step 3: Functions for Meal and Workout Recommendations
def recommend_diverse_foods(abbrev_df):
    cols_to_display = ['Shrt_Desc', 'Water_(g)', 'Energ_Kcal', 'Protein_(g)', 'Carbohydrt_(g)']
    dairy = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('CHEESE|MILK|YOGURT', case=False)].head(5)
    meats = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('BEEF|CHICKEN|TURKEY', case=False)].head(5)
    vegetables = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('CARROT|SPINACH|BROCCOLI', case=False)].head(5)
    grains = abbrev_df[abbrev_df['Shrt_Desc'].str.contains('RICE|OAT|WHEAT', case=False)].head(5)
    return dairy[cols_to_display], meats[cols_to_display], vegetables[cols_to_display], grains[cols_to_display]

def recommend_indian_foods(indian_cuisines_df, goal):
    if goal == 'high_protein':
        return indian_cuisines_df.sort_values(by='proteins', ascending=False).head(10)
    elif goal == 'low_carbs':
        return indian_cuisines_df.sort_values(by='carbohydrates').head(10)
    else:
        return indian_cuisines_df.sort_values(by='calories', ascending=False).head(10)

def generate_weekly_workout_plan(gym_data):
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    workout_plan = {}
    for day in days:
        workouts = gym_data.sample(5)
        workout_plan[day] = workouts[['Title', 'Type', 'BodyPart', 'Equipment', 'Level']].to_dict(orient='records')
    return workout_plan

# Step 4: Get User Inputs
height = float(input("Enter your height in cm: "))
weight = float(input("Enter your weight in kg: "))
age = int(input("Enter your age: "))
gender = input("Enter your gender (male/female): ").lower()
activity_level = input("Enter your activity level (sedentary, light, moderate, active): ").lower()
body_fat_percentage = float(input("Enter your body fat percentage: "))
goal = input("Enter your goal (bulking, cutting, maintaining): ").lower()

# Initialize the Intelligent Agent
agent = FitnessAgent(height, weight, age, gender, activity_level, body_fat_percentage, goal)

# Generate Diet Recommendations
print("\n--- Diet Recommendations ---")
dairy, meats, vegetables, grains, indian_food_recommendations = agent.generate_diet_recommendations()
print("\nDairy:\n", dairy)
print("\nMeats:\n", meats)
print("\nVegetables:\n", vegetables)
print("\nGrains:\n", grains)
print("\nIndian Cuisine Recommendations:\n", indian_food_recommendations)

# Generate and Display Workout Plan
workout_plan = agent.generate_workout_plan()
print("\n--- Weekly Workout Plan ---")
for day, exercises in workout_plan.items():
    print(f"\n{day}:")
    for exercise in exercises:
        print(f"  Exercise: {exercise['Title']}")
        print(f"  Type: {exercise['Type']}")
        print(f"  Target Muscle: {exercise['BodyPart']}")
        print(f"  Equipment: {exercise['Equipment']}")
        print(f"  Level: {exercise['Level']}")
        print("-" * 30)


Enter your height in cm: 180
Enter your weight in kg: 60
Enter your age: 18
Enter your gender (male/female): male
Enter your activity level (sedentary, light, moderate, active): sedentary
Enter your body fat percentage: 10
Enter your goal (bulking, cutting, maintaining): bulking

--- Diet Recommendations ---

Dairy:
           Shrt_Desc  Water_(g)  Energ_Kcal  Protein_(g)  Carbohydrt_(g)
3       CHEESE,BLUE      42.41         353        21.40            2.34
4      CHEESE,BRICK      41.11         371        23.24            2.79
5       CHEESE,BRIE      48.42         334        20.75            0.45
6  CHEESE,CAMEMBERT      51.80         300        19.80            0.46
7    CHEESE,CARAWAY      39.28         376        25.18            3.06

Meats:
                                   Shrt_Desc  Water_(g)  Energ_Kcal  \
129                 EGG,TURKEY,WHL,FRSH,RAW      72.50         171   
327                BABYFOOD,MEAT,TURKEY,STR      80.32         111   
328                 BABYFOOD,M